In [1]:
import pandas as pd
import json
import numpy as np
from haversine import haversine
import os
import datetime
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Zips_for_SOTF_Faith_20190522'

In [2]:
store_list_excel=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Zips_for_SOTF_Faith_20190522/BL_6.7 and 6.21 Zip Codes Updated 5.14.2019.xlsx")
store_list_excel.sheet_names


['BL_6.7 Zips_7 Stores', 'BL_6.21 Zips_15 Stores']

In [3]:
store_list_0607=store_list_excel.parse("BL_6.7 Zips_7 Stores",dtype=str)
store_list_0621=store_list_excel.parse("BL_6.21 Zips_15 Stores",dtype=str)
col_list=store_list_0607.columns.tolist()

print(store_list_0607.shape)
print(store_list_0621.shape)
store_list_0607['G.O.']="2019-06-07"
store_list_0621['G.O.']="2019-06-15"

store_list_0607=store_list_0607[['Store ID','G.O.']]
store_list_0621=store_list_0621[['Store ID','G.O.']]


(7, 8)
(15, 8)


In [4]:
store_list_SFTP=pd.read_csv("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190501-135143-036.txt",
                            dtype=str,sep="|")
store_list_SFTP=store_list_SFTP[['location_id','zip_cd','latitude_meas','longitude_meas']]
store_list_SFTP['latitude_meas']=store_list_SFTP['latitude_meas'].astype(float)
store_list_SFTP['longitude_meas']=store_list_SFTP['longitude_meas'].astype(float)


store_list_SFTP['zip_cd']=store_list_SFTP['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))
store_list_SFTP=store_list_SFTP.rename(columns={"location_id":"Store ID","zip_cd":"store_zip","latitude_meas":"Latitude_meas","longitude_meas":"Longitude_meas"})
store_list_0607=pd.merge(store_list_0607,store_list_SFTP,on="Store ID",how="left")
store_list_0621=pd.merge(store_list_0621,store_list_SFTP,on="Store ID",how="left")

store_list_0607_no_lag_lng=store_list_0607[pd.isnull(store_list_0607['store_zip'])]
store_list_0621_no_lag_lng=store_list_0621[pd.isnull(store_list_0621['store_zip'])]


In [20]:
new_store_update_dict={}

# 5385, 564 SE WASHINGTON BLVD	EASTLAND SHOPPING CENTER	BARTLESVILLE	OK	74006-8231
new_store_update_dict.update({"5385":{"Google_Address":"500 SE Washington Blvd, Bartlesville, OK 74006","Google_Coor":[36.7495305,-95.934563]}})

# 4697, 5910 EASTEX FREEWAY		BEAUMONT	TX	77708
new_store_update_dict.update({"4697":{"Google_Address":"5910 Eastex Fwy, Beaumont, TX 77708","Google_Coor":[30.1285512,-94.1571269]}})

# 5389, 15271 MCGREGOR BOULEVARD	ISLAND PLAZA SHOPPING CENTER	FORT MYERS	FL	33908
new_store_update_dict.update({"5389":{"Google_Address":"15271 McGregor Blvd, Fort Myers, FL 33908","Google_Coor":[26.5187718,-81.9426948]}})

# 4694, 5800 OVERTON RIDGE BOULEVARD	CITY VIEW TOWNE CENTER	FORT WORTH	TX	76132
new_store_update_dict.update({"4694":{"Google_Address":"5800 Overton Ridge Blvd, Fort Worth, TX 76132","Google_Coor":[32.677144,-97.4140692]}})

# 5395, 1308 WEST PATRICK STREET	GOLDEN MILE MARKETPLACE	FREDERICK	MD	21703
new_store_update_dict.update({"5395":{"Google_Address":"1308 W Patrick St, Frederick, MD 21703","Google_Coor":[39.4171998,-77.4570229]}})

# 4695, 6207 SOUTH WESTNEDGE AVENUE	TOYS R US CENTER	PORTAGE	MI	49002
new_store_update_dict.update({"4695":{"Google_Address":"6207 S Westnedge Ave, Portage, MI 49002","Google_Coor":[42.2271103,-85.5868421]}})

# 5372, 10150 DORCHESTER ROAD	DORCHESTER VILLAGE	SUMMERVILLE	SC	29485-8536
new_store_update_dict.update({"5372":{"Google_Address":"10150 Dorchester Rd, Summerville, SC 29485","Google_Coor":[32.9540632,-80.168855]}})

########
########

# 4699, 2500 WEST COMMONWEALTH AVENUE	FREMONT PLAZA	ALHAMBRA	CA	91803
new_store_update_dict.update({"4699":{"Google_Address":"2500 W Commonwealth Ave, Alhambra, CA 91803","Google_Coor":[34.0860875,-118.1510029]}})

# 5396, 2750 NORTH ROBERTS AVENUE	CENTER STAGE OF LUMBERTON	LUMBERTON	NC	28358
new_store_update_dict.update({"5396":{"Google_Address":"2750 N Roberts Ave, Lumberton, NC 28358","Google_Coor":[34.6462145,-79.0104622]}})

# 4696, 4555 16th STREET	TOYS R US CENTER	MOLINE	IL	61265
new_store_update_dict.update({"4696":{"Google_Address":"4555 16th St, Moline, IL 61265","Google_Coor":[41.468197,-90.5084496]}})

# 5393, 5484 ATLANTA HIGHWAY		MONTGOMERY	AL	36109
new_store_update_dict.update({"5393":{"Google_Address":"5484 Atlanta Hwy, Montgomery, AL 36109","Google_Coor":[32.3800155,-86.211963]}})

# 4683, 2415 SAN PABLO DAM ROAD, SUITE 300	SAN PABLO TOWN CENTER	SAN PABLO	CA	94806-3940
new_store_update_dict.update({"4683":{"Google_Address":"2415 San Pablo Dam Rd #300, San Pablo, CA 94806","Google_Coor":[37.9560253,-122.3327502]}})


In [21]:
def allocate_lat(x):
    if x in new_store_update_dict.keys():
        y=new_store_update_dict[x]['Google_Coor'][0]
    else:
        y=np.nan
    return y

def allocate_lng(x):
    if x in new_store_update_dict.keys():
        y=new_store_update_dict[x]['Google_Coor'][1]
    else:
        y=np.nan
    return y

def allocate_zip(x):
    if x in new_store_update_dict.keys():
        y=new_store_update_dict[x]['Google_Address'][-5:]
    else:
        y=np.nan
    return y

In [25]:
store_list_0607['Google_Lat']=store_list_0607['Store ID'].apply(lambda x: allocate_lat(x))
store_list_0607['Google_Lng']=store_list_0607['Store ID'].apply(lambda x: allocate_lng(x))
store_list_0607['Google_Zip']=store_list_0607['Store ID'].apply(lambda x: allocate_zip(x))

store_list_0621['Google_Lat']=store_list_0621['Store ID'].apply(lambda x: allocate_lat(x))
store_list_0621['Google_Lng']=store_list_0621['Store ID'].apply(lambda x: allocate_lng(x))
store_list_0621['Google_Zip']=store_list_0621['Store ID'].apply(lambda x: allocate_zip(x))



In [8]:
store_list_0607['store_zip']=np.where(pd.isnull(store_list_0607['store_zip']), store_list_0607['Google_Zip'],store_list_0607['store_zip'])
store_list_0607['Latitude_meas']=np.where(pd.isnull(store_list_0607['Latitude_meas']), store_list_0607['Google_Lat'],store_list_0607['Latitude_meas'])
store_list_0607['Longitude_meas']=np.where(pd.isnull(store_list_0607['Longitude_meas']), store_list_0607['Google_Lng'],store_list_0607['Longitude_meas'])

store_list_0621['store_zip']=np.where(pd.isnull(store_list_0621['store_zip']), store_list_0621['Google_Zip'],store_list_0621['store_zip'])
store_list_0621['Latitude_meas']=np.where(pd.isnull(store_list_0621['Latitude_meas']), store_list_0621['Google_Lat'],store_list_0621['Latitude_meas'])
store_list_0621['Longitude_meas']=np.where(pd.isnull(store_list_0621['Longitude_meas']), store_list_0621['Google_Lng'],store_list_0621['Longitude_meas'])

store_list_0607=store_list_0607[['Store ID','G.O.','store_zip','Latitude_meas','Longitude_meas']]
store_list_0621=store_list_0621[['Store ID','G.O.','store_zip','Latitude_meas','Longitude_meas']]


In [9]:
store_PST=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                      dtype=str,usecols=['location_id','zip','revenue_flag'])
store_PST=store_PST[store_PST['location_id'].isin(store_list_0607['Store ID'].tolist() + store_list_0621['Store ID'].tolist())]
store_PST=store_PST[store_PST['revenue_flag'].isin(['P','S'])]
print(store_PST.shape)
print(store_PST['location_id'].unique())
store_PST=store_PST.rename(columns={"location_id":"Store ID"})

(176, 3)
['1192' '1286' '1861' '1890' '1918' '1998' '4134' '4527' '4532' '4606']


In [10]:
store_list_0607=pd.merge(store_list_0607,store_PST,on="Store ID",how="left")
store_list_0621=pd.merge(store_list_0621,store_PST,on="Store ID",how="left")


# 0607

In [11]:
store_list_0607_new=store_list_0607[pd.isnull(store_list_0607['zip'])]
store_list_0607_old=store_list_0607[pd.notnull(store_list_0607['zip'])]
print("store_list_0607_old: ",store_list_0607_old.shape )
del store_list_0607_new['zip']
del store_list_0607_new['revenue_flag']

df_new_zip_10_miles=pd.DataFrame()
for index,row in store_list_0607_new.iterrows():
    store_number=row['Store ID']
    store_center=(row['Latitude_meas'],row['Longitude_meas'])
    store_zip=row['store_zip']
    
    df_store=pd.DataFrame({"Store ID":store_number,"zip":store_zip,"Revenue_flag":"store_zip","Distance (mile)":"store_zip"},index=[0])
    
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd])
        if dist<=10:
            df=pd.DataFrame({"Store ID":store_number,"zip":zip_cd,"Revenue_flag":"zips_in_10","Distance (mile)":dist},index=[0])
            df_store=df.append(df_store)
    df_store=df_store.drop_duplicates("zip")
    df_new_zip_10_miles=df_new_zip_10_miles.append(df_store)
    

store_list_0607_old:  (0, 7)


In [12]:
# store_list_0607_old 0 row
output_0607=pd.merge(store_list_0607_new,df_new_zip_10_miles,on="Store ID",how="left")

# 0621

In [13]:
store_list_0621_new=store_list_0621[pd.isnull(store_list_0621['zip'])]
store_list_0621_old=store_list_0621[pd.notnull(store_list_0621['zip'])].rename(columns={"revenue_flag":"Revenue_flag"})

print("store_list_0621_new",store_list_0621_new.shape)
print("store_list_0621_new",store_list_0621_old.shape)

store_list_0621_new (5, 7)
store_list_0621_new (176, 7)


In [14]:
del store_list_0621_new['zip']
del store_list_0621_new['revenue_flag']

df_new_zip_10_miles=pd.DataFrame()
for index,row in store_list_0621_new.iterrows():
    store_number=row['Store ID']
    store_center=(row['Latitude_meas'],row['Longitude_meas'])
    store_zip=row['store_zip']
    
    df_store=pd.DataFrame({"Store ID":store_number,"zip":store_zip,"Revenue_flag":"store_zip","Distance (mile)":"store_zip"},index=[0])
    
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd])
        if dist<=10:
            df=pd.DataFrame({"Store ID":store_number,"zip":zip_cd,"Revenue_flag":"zips_in_10","Distance (mile)":dist},index=[0])
            df_store=df.append(df_store)
    df_store=df_store.drop_duplicates("zip")
    df_new_zip_10_miles=df_new_zip_10_miles.append(df_store)
    
output_0621_new=pd.merge(store_list_0621_new,df_new_zip_10_miles,on="Store ID",how="left")
output_0621_new=output_0621_new[col_list]

In [15]:
store_list_0621_old['Distance (mile)']=np.nan

store_list_0621_old=store_list_0621_old.reset_index()
del store_list_0621_old['index']

for index,row in store_list_0621_old.iterrows():
    store_center=(row['Latitude_meas'],row['Longitude_meas'])
    zip_cd=row['zip']
    dist=haversine(store_center,zip_centers[zip_cd],miles=True)
    # No exception above
    
    store_list_0621_old.loc[index,"Distance (mile)"]=dist
store_list_0621_old=store_list_0621_old[(store_list_0621_old['Revenue_flag']=="P") | (store_list_0621_old['Distance (mile)']<=20)]    

In [16]:
output_0621=store_list_0621_old.append(output_0621_new)
output_0621=output_0621[col_list]

In [18]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q2/Planner_Requests/Zips_for_SOTF_Faith_20190522/BL_6.7 and 6.21 Zip Codes Updated 5.14.2019_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
output_0607.to_excel(writer,'BL_6.7 Zips_7 Stores',index=False)
output_0621.to_excel(writer,'BL_6.21 Zips_15 Stores',index=False)
writer.save()
